In [ ]:
import pandas as pd
import numpy as np 
import random
from sklearn.decomposition import PCA,KernelPCA
from scipy.spatial.distance import jaccard
from sklearn.manifold import TSNE
import umap
#import umap.umap_ as umap
from math import*
import plotly.express as px
import ipywidgets as widgets
import plotly.offline as py
from ipywidgets import interact,Layout,GridspecLayout, Button
from IPython.display import display, clear_output
from tqdm import tqdm
from scipy import sparse
from scipy.sparse import csr_matrix
py.init_notebook_mode(connected=True)

In [ ]:
def get_user_selection(a):
        with out:
            clear_output()
            select_country = dropbox1.value
            country_list= [str(x).split('.')[0] for x in select_country]
            
            country_name = [str(x).split('(')[0] for x in country_list]
            country= ','.join(sorted(country_name))
            print("country/countries selected: ",country)
            
            sample_size = dropbox3.value
            timestamp = dropbox4.value
            region = dropbox5.value
            country_len = len(select_country)
            sample_len = sample_size//country_len

            col_name =  ['Week','Host','Date','Country','PangolinClade','EPI_ID','x','y']
            #print(select_country)
            timestamp_dict = {'oct-april_2023':'Last_6months','april2022-april2023':'Last_12months',
                              'april2021-april2024':'Last_24months','jan-jun_2020':'jan-jun_2020', 'jan-dec_2020':'jan-dec_2020',
                              'jul-dec_2020':'jul-dec_2020','jan-dec_2021':'jan-dec_2021','jan-jun_2021':'jan-jun_2021','jul-dec_2021':'jul-dec_2021',
                              'jan-jun_2022':'jan-jun_2022','jul-dec_2022':'jul-dec_2022', 'jan-dec_2022':'jan-dec_2022', 'jan-jun_2023':'jan_jun_2023','whole':'whole'}
            
            get_time_val = timestamp_dict[timestamp]
            
            def pick_timeline(t):
                if t == 'Last_6months':
                    idx=(meta[:,2]> '2022-10-17') & (meta[:,2]<='2023-04-17')
                    result= np.where(idx)[0]
                    return result
                   
                elif t == 'Last_12months':
                    idx=(meta[:,2]> '2022-04-17') & (meta[:,2]<='2023-04-17')
                    result=  np.where(idx)[0]
                    return result
                   
                elif t == 'Last_24months':
                    idx=(meta[:,2]> '2021-04-17') & (meta[:,2]<='2023-04-17')
                    result = np.where(idx)[0]
                    return result

                elif t == 'jan-jun_2020':
                    idx=(meta[:,2]> '2020-01-01') & (meta[:,2]<='2020-12-31')#2020-06-30
                    result= np.where(idx)[0]
                    return result
                   
                elif t == 'jul-dec_2020':
                    print("hello")
                    idx=(meta[:,2]> '2020-07-01') & (meta[:,2]<='2020-12-31')
                    result=  np.where(idx)[0]
                    return result

                elif t == 'jan-dec_2020':
                    print("hello")
                    idx=(meta[:,2]> '2020-01-01') & (meta[:,2]<='2020-12-31')
                    result=  np.where(idx)[0]
                    return result
                   
                elif t == 'jan-jun_2021':
                    idx=(meta[:,2]> '2021-01-01') & (meta[:,2]<='2021-06-30') #2021-06-30
                    result = np.where(idx)[0]
                    return result

                elif t == 'jan-dec_2021':
                    idx=(meta[:,2]> '2021-01-01') & (meta[:,2]<='2021-12-31') #2021-06-30
                    result = np.where(idx)[0]
                    return result
                  
                elif t == 'jul-dec_2021':
                    idx=(meta[:,2]> '2021-07-01') & (meta[:,2]<='2021-12-31')
                    result= np.where(idx)[0]
                    return result
            
                elif t == 'jan-jun_2022':
                    idx=(meta[:,2]> '2022-01-01') & (meta[:,2]<='2022-12-31') # 2022-06-30
                    result= np.where(idx)[0]
                    return result

                elif t == 'jul-dec_2022':
                    idx=(meta[:,2]> '2022-07-01') & (meta[:,2]<='2022-12-31')
                    result= np.where(idx)[0]
                    return result

                elif t == 'jan-dec_2022':
                    idx=(meta[:,2]> '2022-01-01') & (meta[:,2]<='2022-12-31') #2021-06-30
                    result = np.where(idx)[0]
                    return result

                elif t == 'jan-jun_2023':
                    idx=(meta[:,2]> '2023-01-01') & (meta[:,2]<='2023-06-30')
                    result= np.where(idx)[0]
                    return result
                    
                elif t == 'whole':
                    idx=(meta[:,2]>= '2020-01-01') & (meta[:,2]<='2022-06-30')
                    result= np.where(idx)[0]
                    return result
                   
            
                   
            
######### reading the file/files, perform random sampling based on samplesize and putting them into dataframe######################3
            checkpoint =[]
            metadata = []
            for filename in tqdm(select_country,desc= "loading file"):
                new_list = []
                with open("meta_new/%s.tsv" % filename.split("(")[0]) as fin:
                    for line in fin:
                        line = line.rstrip().split("\t")
                        new_list.append(line)
                meta= np.array(new_list)
                #print(meta.shape)
                idx_meta = pick_timeline(get_time_val)
                if idx_meta.size == 0:
                    print("\033[1;31m No datapoints found for specified criteria  \n")
                    return
                

                index_meta = list(idx_meta)
                #print(len(index_meta))
                np.random.seed(42)
                sample_index_meta = np.random.choice(index_meta,min(len(index_meta),sample_len),replace =False)
                #print(sample_index_meta)
                with open("data_npz_new/%s.npz" % filename.split("(")[0], 'rb') as f:
                    data_matrix = sparse.load_npz(f)                    
                    #np.random.seed(42)
                    m = data_matrix[sample_index_meta]
                    X_m = m.toarray()
                    z_m = meta[sample_index_meta, :]
                    checkpoint.append(X_m)
                    metadata.append(z_m)
            X =np.concatenate(checkpoint)
            z = np.concatenate(metadata)
            #print(z)
            if dropbox6.value == 'All':
                
                X =np.concatenate(checkpoint)
                z = np.concatenate(metadata)

            elif dropbox6.value == 'Alpha':
                
                index_meta = np.where((z == 'B.1.1.7'))[0]
                #print(index_meta)
                axis = 0
                z = np.take(z, index_meta, axis)
                X= np.take(X, index_meta,axis)
                
            elif dropbox6.value == 'Beta':
                
                index_meta = np.where((z == 'B.1.351'))[0]
                #print(index_meta)
                axis = 0
                z = np.take(z, index_meta, axis)
                X= np.take(X, index_meta,axis)
            elif dropbox6.value == 'Gamma':
                
                index_meta = np.where((z == 'P.1'))[0]
                #print(index_meta)
                axis = 0
                z = np.take(z, index_meta, axis)
                X= np.take(X, index_meta,axis)
            elif dropbox6.value == 'Delta':
                
                index_meta = np.where((z == 'B.1.617')|(z == 'B.1.617.1')|(z=='B.1.617.2')|(z=='B.1.617.3'))[0]
                #print(index_meta)
                axis = 0
                z = np.take(z, index_meta, axis)
                X= np.take(X, index_meta,axis)
                #print(len(z))
            elif dropbox6.value == 'Omicron':
                index_meta = np.where((z== 'B.1.1.529')| (z == 'BA.1')|(z== 'BA.1.1')|(z== 'BA.2')|(z== 'BA.3')|(z== 'BA.4')|(z== 'BA.5'))[0] #In BOV Check once
                #index_meta = np.where((z == 'B.1.1.529')|(z== 'BA.1')|(z== 'BA.1.1')|(z== 'BA.2')|(z== 'BA.3'))[0]
                #print(index_meta)
                axis = 0
                z = np.take(z, index_meta, axis)
                X= np.take(X, index_meta,axis)

            elif dropbox6.value == 'VOC_All':
                
                index_meta = np.where((z == 'BA.1')|(z== 'B.1.1.7')|(z== 'BA.2')|(z== 'BA.3')|(z== 'BA.4')|(z== 'BA.5')|(z== 'B.1.617.2')|(z== 'P.1')|(z== 'B.1.351'))[0]
                #print(index_meta)
                #
                axis = 0
                z = np.take(z, index_meta, axis)
                X= np.take(X, index_meta,axis) 
                
            #print("Final dimension of feature vectors:",X.shape)

            ######## function to pick functional genomic region of interest#####################################
            def pick_region(v):
                if v == 'whole':
                    whole = X
                    return whole
                elif v == 'spike':
                    spike = np.array(X[:,55034:65385]).astype(np.int8)
                    return spike
                elif v == 'membrane':
                    membrane = np.array(X[:,68597:70341]).astype(np.int8)
                    return membrane
                elif v == 'envelope':
                    envelope = np.array(X[:,67849:68450]).astype(np.int8)
                    return envelope
                elif v == 'nucleocapsid':
                    nucleocapsid = np.array(X[:,73564:77025]).astype(np.int8)
                    return nucleocapsid
                return val
                
            
            X = pick_region(dropbox5.value)
            print("resulting vector after filtering",X.shape)

           
            print("Final dimension of feature vectors:", X.shape)
            if X.shape[0] == 0:
                print("\033[1;31m No datapoints found for specified criteria  \n")
                return
            #np.savetxt("foo.csv", val, fmt="%d", delimiter=",")
            
            flatten_arr = np.ravel(X)
            result = np.all(X==flatten_arr[0])
            if result :
                print("\033[1;31m There are no variation found in the specified selection, all datapoints are similar  \n")
                return
            
            
            
            chunk_size = 2000
            total_size = X.shape[0]
            min_size = (min(total_size,sample_size))
            if min_size > 2000:
                if not min_size % 2000 == 0:
                    get_val = (min_size//chunk_size)+1
                else:
                    get_val = sample_size//chunk_size
                
            else:
                get_val = 1
            
            
            ptype = "VE"
            vistype = dropbox2.value
            plotname =  ",".join(str(x).split('(')[0] for x in country_list)+";"+ptype+";"+vistype
            colors_data = pd.read_csv("Color_map.txt",sep='\t')
            #group_color = colors_data.set_index('Clade')['RGB'].to_dict()
            #group_color = {'BA.1':'blue','BA.2':'red','BA.1.1':'green','BA.3':'purple',
                                       #'B.1.1.529':'orange'}
            group_color = {'BA.4':'yellow','BA.2':'pink','BA.5':'violet','BA.3':'brown','B.1.351':'purple','P.1':'orange',
                                       'BA.1':'green','B.1.617.2':'red','B.1.1.7':'blue','India':'green','South Africa':'blue','Brazil':'orange',
                                      'United Kingdom': 'red'}
            if dropbox2.value =='PCA':
                pca = PCA(n_components=2)
                pca_matrix=[]
                for i in tqdm(range(0, get_val),desc="Performing dimensionality reduction on feature vectors"):
                    X_val = X[i*chunk_size : (i+1)*chunk_size]
                    X_embed_partial= pca.fit_transform(X_val)
                    pca_matrix.append(X_embed_partial)
                components = np.concatenate(pca_matrix)
                #components = pca.fit_transform(X)
            

                def print_plot_pca(grpval):
                    map2 ={}
                    data = []

                    for i in range(0,len(z)):
                        map2[i] = z[i]
                    for k,v in enumerate(components):
                        meta = map2[k].tolist()
                        comp = v.tolist()
                        val = meta + comp
                        data.append(val)

                    df1 =pd.DataFrame(data,columns = col_name)
                    df1['counts'] = df1.PangolinClade.map(df1.PangolinClade.value_counts())
                    df1= df1.sort_values(by=['counts'], ascending=False)
                    #print(df1)
                    df1["Week"] = df1["Week"].astype(float) #convert back to numeric
                    labels = {
                        str(i): f"PC {i+1} ({var:.1f}%)"
                        for i, var in enumerate(pca.explained_variance_ratio_ * 100)
                        }
                    fig = px.scatter(df1, x='x', y='y',color=grpval,color_discrete_map=group_color,hover_name='EPI_ID', hover_data= {'x' : False,'y' : False,'Week': True,'Host': True,'Date': True,'Country': True,'PangolinClade':True},labels=labels)

                    fig.update_traces(marker=dict(size=10),
                                              selector=dict(mode='markers'))
                    fig.update_layout(
                                title=plotname, #plot title 
                                paper_bgcolor='white',
                                plot_bgcolor='white',
                                showlegend = True,
                                font=dict(
                                    family="Courier New, monospace",
                                    size=18,
                                    color="black"
                                )
                            )
                    fig.update_xaxes(title_text=' ',
                                   showgrid=True,
                                   zeroline=True,
                                   showline=True,
                                   gridcolor='#bdbdbd',
                                   gridwidth=2,
                                   zerolinecolor='#969696',
                                   zerolinewidth=4,
                                   linecolor='#969696',
                                   linewidth=4)
                    fig.update_yaxes(title_text=' ',
                                   showgrid=True,
                                   zeroline=True,
                                   showline=True,
                                   gridcolor='#bdbdbd',
                                   gridwidth=2,
                                   zerolinecolor='#969696',
                                   zerolinewidth=4,
                                   linecolor='#969696',
                                   linewidth=4)

                    py.iplot(fig)
                    py.plot(fig, filename='check.html')
                im = interact(print_plot_pca,grpval= ['Country','PangolinClade','Week'])
                im.widget.children[0].description = 'Color code by : '
                im.widget.children[0].style = {'description_width': 'initial'}
                display(im)



                    
                
            elif dropbox2.value =='t-SNE':
                tsne = TSNE(n_components=2, perplexity = 30)
                tsne_matrix=[]
                for i in tqdm(range(0, get_val),desc="Performing dimensionality reduction on feature vectors"):
                    X_val = X[i*chunk_size : (i+1)*chunk_size]
                    X_embed_partial= tsne.fit_transform(X_val)
                    tsne_matrix.append(X_embed_partial)
                components_tsne = np.concatenate(tsne_matrix)

                def print_plot_tsne(grpval):
                    map2 ={}
                    data = []
                
                    for i in range(0,len(z)):
                        map2[i] = z[i]
                    for k,v in enumerate(components_tsne):
                        meta = map2[k].tolist()
                        comp = v.tolist()
                        val = meta + comp
                        data.append(val)

                    df1 =pd.DataFrame(data,columns = col_name)
                    df1['counts'] = df1.PangolinClade.map(df1.PangolinClade.value_counts())
                    df1= df1.sort_values(by=['counts'], ascending=False)
                    #print(df1)
                    df1["Week"] = df1["Week"].astype(float) #convert back to numeric
                    fig = px.scatter(df1, x='x', y='y',color=grpval,color_discrete_map=group_color,hover_name='EPI_ID', hover_data= {'x' : False,'y' : False,'Week': True,'Host': True,'Date': True,'Country': True,'PangolinClade':True})

                    fig.update_traces(marker=dict(size=10),
                                              selector=dict(mode='markers'))
                    fig.update_layout(
                                title=plotname, # plot title 
                                paper_bgcolor='white',
                                plot_bgcolor='white',
                                showlegend = True,
                                font=dict(
                                    family="Courier New, monospace",
                                    size=18,
                                        color="black"
                                )
                            )
                    fig.update_xaxes(title_text=' ',
                                   showgrid=True,
                                   zeroline=True,
                                   showline=True,
                                   gridcolor='#bdbdbd',
                                   gridwidth=2,
                                   zerolinecolor='#969696',
                                   zerolinewidth=4,
                                   linecolor='#969696',
                                   linewidth=4)
                    fig.update_yaxes(title_text=' ',
                                   showgrid=True,
                                   zeroline=True,
                                   showline=True,
                                   gridcolor='#bdbdbd',
                                   gridwidth=2,
                                   zerolinecolor='#969696',
                                   zerolinewidth=4,
                                   linecolor='#969696',
                                   linewidth=4)

                    py.iplot(fig)
                    py.plot(fig, filename='check.html')
                im = interact(print_plot_tsne,grpval= ['Country','PangolinClade','Week'])
                im.widget.children[0].description = 'Color code by : '
                im.widget.children[0].style = {'description_width': 'initial'}
                display(im)
  
            elif dropbox2.value == 'UMAP':
                umap_2d = umap.UMAP(n_components=2, init='random', random_state=0)
                umap_matrix=[]
                for i in tqdm(range(0, get_val),desc="Performing dimensionality reduction on feature vectors"):
                    X_val = X[i*chunk_size : (i+1)*chunk_size]
                    X_embed_partial= umap_2d.fit_transform(X_val)
                    umap_matrix.append(X_embed_partial)
                components_umap = np.concatenate(umap_matrix)

                def print_plot_umap(grpval):
                    map2 ={}
                    data = []
                    for i in range(0,len(z)):
                        map2[i] = z[i]
                    for k,v in enumerate(components_umap):
                        meta = map2[k].tolist()
                        comp = v.tolist()
                        val = meta + comp
                        data.append(val)

                    df1 =pd.DataFrame(data,columns = col_name)
                    df1['counts'] = df1.PangolinClade.map(df1.PangolinClade.value_counts())
                    df1= df1.sort_values(by=['counts'], ascending=False)
                    df1["Week"] = df1["Week"].astype(float) #convert back to numeric
                    fig = px.scatter(df1, x='x', y='y',color=grpval,color_discrete_map=group_color,hover_name='EPI_ID', hover_data= {'x' : False,'y' : False,'Week': True,'Host': True,'Date': True,'Country': True,'PangolinClade':True})

                    fig.update_traces(marker=dict(size=10),
                                              selector=dict(mode='markers'))
                    fig.update_layout(
                                title=plotname, #plot title 
                                paper_bgcolor='white',
                                plot_bgcolor='white',
                                showlegend = True,
                                font=dict(
                                    family="Courier New, monospace",
                                    size=18,
                                    color="black"
                                )
                            )
                    fig.update_xaxes(title_text=' ',
                                   showgrid=True,
                                   zeroline=True,
                                   showline=True,
                                   gridcolor='#bdbdbd',
                                   gridwidth=2,
                                   zerolinecolor='#969696',
                                   zerolinewidth=4,
                                   linecolor='#969696',
                                   linewidth=4)
                    fig.update_yaxes(title_text=' ',
                                   showgrid=True,
                                   zeroline=True,
                                   showline=True,
                                   gridcolor='#bdbdbd',
                                   gridwidth=2,
                                   zerolinecolor='#969696',
                                   zerolinewidth=4,
                                   linecolor='#969696',
                                   linewidth=4)

                    py.iplot(fig)
                    py.plot(fig, filename='check.html')
                im = interact(print_plot_umap,grpval= ['Country','PangolinClade','Week'])
                im.widget.children[0].description = 'Color code by : '
                im.widget.children[0].style = {'description_width': 'initial'}
                display(im)
                
            elif dropbox2.value == 'KPCA-cosine':
                n_princomp=2
                KPCA = KernelPCA(n_princomp,kernel='cosine')
                matrix=[]
                for i in tqdm(range(0, get_val),desc="Performing dimensionality reduction on feature vectors"):
                    X_embed_partial=KPCA.fit_transform(X[i*chunk_size : (i+1)*chunk_size])
                    matrix.append(X_embed_partial)
                components = np.concatenate(matrix)
                def print_plot_kpca_cosine(grpval):
                    map2 ={}
                    data = []
                    for i in range(0,len(z)):
                        map2[i] = z[i]
                    for k,v in enumerate(components):
                        meta = map2[k].tolist()
                        comp = v.tolist()
                        val = meta + comp
                        data.append(val)

                    df1 =pd.DataFrame(data,columns = col_name)
                    df1['counts'] = df1.PangolinClade.map(df1.PangolinClade.value_counts())
                    df1= df1.sort_values(by=['counts'], ascending=False)
                    df1["Week"] = df1["Week"].astype(float) #convert back to numeric
               
                    fig = px.scatter(df1, x='x', y='y',color=grpval,color_discrete_map=group_color,hover_name='EPI_ID', hover_data= {'x' : False,'y' : False,'Week': True,'Host': True,'Date': True,'Country': True,'PangolinClade':True})

                  
                    fig.update_traces(marker=dict(size=10),
                                              selector=dict(mode='markers'))
                    fig.update_layout(
                                title=plotname, #plot title 
                                paper_bgcolor='white',
                                plot_bgcolor='white',
                                showlegend = True,
                                font=dict(
                                    family="Courier New, monospace",
                                    size=18,
                                    color="black"
                                )
                            )
                    fig.update_xaxes(title_text=' ',
                                   showgrid=True,
                                   zeroline=True,
                                   showline=True,
                                   gridcolor='#bdbdbd',
                                   gridwidth=2,
                                   zerolinecolor='#969696',
                                   zerolinewidth=4,
                                   linecolor='#969696',
                                   linewidth=4)
                    fig.update_yaxes(title_text=' ',
                                   showgrid=True,
                                   zeroline=True,
                                   showline=True,
                                   gridcolor='#bdbdbd',
                                   gridwidth=2,
                                   zerolinecolor='#969696',
                                   zerolinewidth=4,
                                   linecolor='#969696',
                                   linewidth=4)

                    py.iplot(fig)
                    py.plot(fig, filename='check.html')
                im = interact(print_plot_kpca_cosine,grpval= ['Country','PangolinClade','Week'])
                im.widget.children[0].description = 'Color code by : '
                im.widget.children[0].style = {'description_width': 'initial'}
                display(im)
                
            elif dropbox2.value == 'KPCA-jaccard':
                
                def jaccard_distance(list1, list2):
                    intersection = len(list(set(list1).intersection(list2)))
                    union = (len(list1) + len(list2)) - intersection
                    return float(intersection) / union
                    

                np.seterr(all='ignore')
                X_mod=X 
                N=X_mod.shape[0]
                A=np.zeros((N,N))
                #print(A)
                for i in range(0,N):
                    for j in range(0,N):
                        A[i][j]= jaccard(X_mod[i],X_mod[j])

                #print(A)
                n_princomp=2
                KPCA = KernelPCA(n_princomp,kernel='precomputed')
                components = KPCA.fit_transform(A)
                def print_plot_kpca_jaccard(grpval):
                    map2 ={}
                    data = []
                    for i in range(0,len(z)):
                        map2[i] = z[i]
                    for k,v in enumerate(components):
                        meta = map2[k].tolist()
                        comp = v.tolist()
                        val = meta + comp
                        data.append(val)

                    df1 =pd.DataFrame(data,columns = col_name)
                    df1["Week"] = df1["Week"].astype(float) #convert back to numeric
                    fig = px.scatter(df1, x='x', y='y',color=grpval,hover_name='EPI_ID', hover_data= {'x' : False,'y' : False,'Week': True,'Host': True,'Date': True,'Country': True,'PangolinClade':True})

                              
                    fig.update_traces(marker=dict(size=10),
                                              selector=dict(mode='markers'))
                    fig.update_layout(
                                title=plotname, #plot title 
                                paper_bgcolor='white',
                                plot_bgcolor='white',
                                showlegend = True,
                                font=dict(
                                    family="Courier New, monospace",
                                    size=18,
                                    color="black"
                                )
                            )
                    fig.update_xaxes(title_text=' ',
                                   showgrid=True,
                                   zeroline=True,
                                   showline=True,
                                   gridcolor='#bdbdbd',
                                   gridwidth=2,
                                   zerolinecolor='#969696',
                                   zerolinewidth=4,
                                   linecolor='#969696',
                                   linewidth=4)
                    fig.update_yaxes(title_text=' ',
                                   showgrid=True,
                                   zeroline=True,
                                   showline=True,
                                   gridcolor='#bdbdbd',
                                   gridwidth=2,
                                   zerolinecolor='#969696',
                                   zerolinewidth=4,
                                   linecolor='#969696',
                                   linewidth=4)

                    py.iplot(fig)
                im = interact(print_plot_kpca_jaccard,grpval= ['Country','PangolinClade','Week'])
                im.widget.children[0].description = 'Color code by : '
                im.widget.children[0].style = {'description_width': 'initial'}
                display(im)

In [ ]:
country_data = pd.read_csv("country_data_analysis.txt",sep='\t')
d = country_data.set_index('Country')['Samples'].to_dict()

            
out = widgets.Output()


dropbox1 = widgets.SelectMultiple(options=['{}({})'.format(k,v) for k,v in d.items()],description='Country:', 
                        disabled=False)

dropbox1.layout.height= '200px'

dropbox5 = widgets.Dropdown(
    options=['envelope','membrane','nucleocapsid','spike','whole'],
    value='whole',
    rows=5,
    description='Genomic region of interest:',
    style={'description_width': 'initial'},
    disabled=False
)

dropbox3 = widgets.BoundedIntText(
    value=10000,
    max=15000,
    step=500,
    description='Datapoints on canvas:',
    style={'description_width': 'initial'},
    disabled=False
)

dropbox4 = widgets.Dropdown(
    options=['oct-april_2023','april2022-april2023','april2021-april2024','jan-dec_2020','jan-jun_2020','jul-dec_2020','jan-dec_2021','jan-jun_2021','jul-dec_2021', 'jan-dec_2022', 'jan-jun_2022','jul-dec_2022','jan-jun_2023','whole'],
    value='oct-april_2023',
    rows=4,
    #layout={'width': 'max-content'},
    description='Timestamp:',
    style={'description_width': 'initial'},
    disabled=False
)
dropbox2 = widgets.Dropdown(
    options=['PCA','t-SNE','UMAP','KPCA-cosine','KPCA-jaccard'],
    value='PCA',
    description='Type of plot:',
    style={'description_width': 'initial'},
    disabled=False
)

dropbox6 = widgets.Dropdown(
    options=['Alpha','Beta','Gamma','Delta','Omicron','VOC_All','All'],
    value= 'All',
    rows=5,
    description='Select Lineage:',
    style={'description_width': 'initial'
          },
    disabled=False
)

def dropdown_eventhandler(change):
    with out:
        clear_output()
        if change['type'] == 'change' and change['name'] == 'value':
            country_list= [str(x).split('.')[0] for x in change.new]
            country_name = [str(x).split('(')[0] for x in country_list]
            country= ','.join(sorted(country_name))
            print("\033[1;32m Country/Countries selected: ", country)

box_layout = widgets.Layout(display = 'flex',
                flex_flow='column',
                align_items='stretch',
                #width='60%',
                #height = '60%'
                 )
# Button to click
select_button = widgets.Button(
    description='Run',
    disabled=False,
    layout = box_layout,
    button_style='success',
)
dropbox1.observe(dropdown_eventhandler, names='value')

select_button.on_click(get_user_selection)

grid = GridspecLayout(5, 3, height='300px', align_items = 'baseline', overflow='auto')
grid[0, 1] = dropbox6
grid[:, 0] = dropbox1
grid[1, 1] = dropbox4
grid[0, 2] = dropbox3
grid[3, 1] = select_button
grid[1,2] = dropbox2
grid[2,1] = dropbox5

display(grid,out)
dropbox1.observe(dropdown_eventhandler, names='value')